<a href="https://colab.research.google.com/github/PhamBaVuong/PhamBaVuong_DATN/blob/main/Routing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install polyline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import requests
import folium
import polyline

In [ ]:
url = "http://router.project-osrm.org/route/v1/driving/13.388860,52.517037;13.397634,52.529407;13.428555,52.523219?overview=false"
r = requests.get(url)
res = r.json()
res

{'code': 'Ok',
 'routes': [{'legs': [{'steps': [],
     'summary': '',
     'weight': 260.8,
     'duration': 259.6,
     'distance': 1884},
    {'steps': [],
     'summary': '',
     'weight': 365.7,
     'duration': 365.7,
     'distance': 2839.2}],
   'weight_name': 'routability',
   'weight': 626.5,
   'duration': 625.3,
   'distance': 4723.2}],
 'waypoints': [{'hint': 'hAEKgEwlmYUXAAAABQAAAAAAAAAgAAAAfXRPQdLNK0AAAAAAsPePQQsAAAADAAAAAAAAABAAAAAb7wAA_kvMAKlYIQM8TMwArVghAwAA7wobj63C',
   'distance': 4.231666,
   'name': 'Friedrichstraße',
   'location': [13.388798, 52.517033]},
  {'hint': 'g_jcgWl6iIcGAAAACgAAAAAAAAB3AAAA5JSNQJ0fwkAAAAAAGjmEQgYAAAAKAAAAAAAAAHcAAAAb7wAAfm7MABiJIQOCbswA_4ghAwAAXwUbj63C',
   'distance': 2.795167,
   'name': 'Torstraße',
   'location': [13.39763, 52.529432]},
  {'hint': 'ZywYgP___38fAAAAUQAAACYAAAAeAAAAsowKQkpQX0Lf6yZCvsQGQh8AAABRAAAAJgAAAB4AAAAb7wAASufMAOdwIQNL58wA03AhAwMAvxAbj63C',
   'distance': 2.226595,
   'name': 'Platz der Vereinten Nationen',
   

In [2]:
def get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):
    
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://router.project-osrm.org/route/v1/driving/"
    r = requests.get(url + loc) 
    if r.status_code!= 200:
        return {}
  
    res = r.json()   
    routes = polyline.decode(res['routes'][0]['geometry'])
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']
    
    out = {'route':routes,
           'start_point':start_point,
           'end_point':end_point,
           'distance':distance
          }

    return out

In [25]:
pickup_lat, pickup_lon, dropoff_lat, dropoff_lon = 10.85109302207132, 106.66395888608214 , 10.767112992327842, 106.69739558504001
test_route = get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
test_route

{'route': [(10.85113, 106.66389),
  (10.8505, 106.66386),
  (10.84197, 106.67655),
  (10.83859, 106.67551),
  (10.83512, 106.67561),
  (10.82649, 106.67994),
  (10.82275, 106.67889),
  (10.81449, 106.67861),
  (10.80837, 106.67363),
  (10.80543, 106.67813),
  (10.79954, 106.67492),
  (10.79933, 106.67756),
  (10.79417, 106.67785),
  (10.79212, 106.68128),
  (10.78147, 106.69294),
  (10.77501, 106.6868),
  (10.77161, 106.69299),
  (10.76704, 106.69591),
  (10.76762, 106.69693),
  (10.76713, 106.69742)],
 'start_point': [10.851126, 106.663889],
 'end_point': [10.767125, 106.697418],
 'distance': 12866.9}

In [26]:
def get_map(route):
    
    m = folium.Map(location=[(route['start_point'][0] + route['end_point'][0])/2, 
                             (route['start_point'][1] + route['end_point'][1])/2], 
                   zoom_start=12)

    folium.PolyLine(
        route['route'],
        weight=8,
        color='blue',
        opacity=0.6
    ).add_to(m)

    folium.Marker(
        location=route['start_point'],
        icon=folium.Icon(icon='play', color='green')
    ).add_to(m)

    folium.Marker(
        location=route['end_point'],
        icon=folium.Icon(icon='stop', color='red')
    ).add_to(m)

    return m

In [27]:
get_map(test_route)

In [9]:
freegeoip = "http://freegeoip.net/json"
geo_r = requests.get(freegeoip)
geo_json = geo_r.json()

user_postition = [geo_json["latitude"], geo_json["longitude"]]

print(user_postition)

JSONDecodeError: ignored

In [11]:
pip install geocoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98 kB 3.2 MB/s 


In [12]:
import geocoder
g = geocoder.ip('me')
print(g.latlng)

[41.2619, -95.8608]


In [16]:
pip install winrt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement winrt (from versions: none)
ERROR: No matching distribution found for winrt


In [17]:
import winrt.windows.devices.geolocation as wdg, asyncio

async def getCoords():
        locator = wdg.Geolocator()
        pos = await locator.get_geoposition_async()
        return [pos.coordinate.latitude, pos.coordinate.longitude]
    
def getLoc():
    return ascyncio.run(getCoords())

ModuleNotFoundError: ignored

In [20]:
# importing geopy library
from geopy.geocoders import Nominatim
 
# calling the Nominatim tool
loc = Nominatim(user_agent="GetLoc")
 
# entering the location name
getLoc = loc.geocode("current location")
 
# printing address
print(getLoc.address)
 
# printing latitude and longitude
print("Latitude = ", getLoc.latitude, "\n")
print("Longitude = ", getLoc.longitude)

AttributeError: ignored

In [23]:
pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 985 kB 5.1 MB/s 
     |████████████████████████████████| 358 kB 54.8 MB/s 
     |████████████████████████████████| 140 kB 52.3 MB/s 
     |████████████████████████████████| 58 kB 5.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.


In [24]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait

def getLocation():
    chrome_options = Options()
    chrome_options.add_argument("--use-fake-ui-for-media-stream")
    timeout = 20
    driver = webdriver.Chrome(chrome_options=chrome_options)
    driver.get("https://mycurrentlocation.net/")
    wait = WebDriverWait(driver, timeout)
    longitude = driver.find_elements_by_xpath('//*[@id="longitude"]')
    longitude = [x.text for x in longitude]
    longitude = str(longitude[0])
    latitude = driver.find_elements_by_xpath('//*[@id="latitude"]')
    latitude = [x.text for x in latitude]
    latitude = str(latitude[0])
    driver.quit()
    return (latitude,longitude)
print (getLocation())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: use options instead of chrome_options
  if __name__ == '__main__':


WebDriverException: ignored